# Data Analytics / Data Science - Datathon 2024

## Libraries

Import commands of libraries and functions that are going to be used for this notebook

In [0]:
# Loading Libraries for the notebook
from pyspark.sql.functions import *
import datetime
from pyspark.sql.window import Window


# UDFS

UDFS, are user defined functions, In the cell below is the definition of the functions that we created that are going to be used in this notebook.

In [0]:

# Function to Calculate Distance in Kilometers in a straight line between two points.

'''
CAL_LAT_LONG_DIST(df,lat1,long1,lat2,long2):
df = dataframe when we want to add the columns of "distance_in_kms"
lat1 = Latitued of Place 1
long1 = Longitude of Place 1
lat 2 = Latitude of Place 2
long2 = Longitued of Place 2

Outputs: DataFrame with a new column named = "distance_in_kms". The number represent the distance in KMS between the two points for each row.
'''

def cal_lat_log_dist(df, lat1, long1, lat2, long2):
        df = df.withColumn('distance_in_kms' , \
            round((acos((sin(radians(col(lat1))) * sin(radians(col(lat2)))) + \
                   ((cos(radians(col(lat1))) * cos(radians(col(lat2)))) * \
                    (cos(radians(long1) - radians(long2))))
                       ) * lit(6371.0)), 4))
        return df

## Loading Data

The data we would use for our use case is GDELT poject database. That consists in two tables: GDELT_EVENTS and GKG.

Additionaly we would use two more tables:

* PORT_LOCATIONS_DIM:

Is a table that locates differnet ports around the world with their country, state or province and  coordinates (longitude and latitude)

* CAMEO_DICTRIONARY:

 Is a table that helps to identify the descriptions of CAMEO EVENTS codes. To correctly identify the nature of the events


 All this tables came from the BRONZE database. Wich contains all tables in their raw forms

In [0]:
# Loading Data from BRONZE database

GDELT_EVENTS = spark.sql("SELECT * FROM BRONZE.GDELT_EVENTS")
PORT_LOCATIONS_DIM = spark.sql("SELECT * FROM BRONZE.PORTS_DICTIONARY")
CAMEO_DICTIONARY = spark.sql("SELECT * FROM BRONZE.CAMEO_DICTIONARY")
GKG = spark.sql("SELECT * FROM BRONZE.GDELT_GKG")

## Cleaning PORT_LOCATIONS_DIM

PORT_LOCATIONS_DIM Requieres some cleaning before using it.

1. Filter Ports with no coordinates 
2. Eliminate blank spaces in LATITUDE and LONGITUDE
3. Extract the Orientation of the coordinates
4. Correctly describe Latitude and Longitude with it's corresponding orientation
5. Select columns of interest: "COUNTRY","PORT","LATITUDE_CORRECTED","LONGITUDE_CORRECTED" for a consist and clean table

In [0]:
# Claeaning RAW data from PORT_LOCATIONS

PORT_LOCATIONS_DIM_CLEANED = (
PORT_LOCATIONS_DIM
.filter("LATITUDE IS NOT NULL") #Filter for Latitud is nos null
.filter("LONGITUDE IS NOT NULL") #Filter for Longitud is nos null
.withColumn("LATITUDE", regexp_replace(col("LATITUDE")," ","")) #Eliminate black spaces in LATITUD column
.withColumn("LONGITUDE", regexp_replace(col("LONGITUDE")," ","")) #Eliminate black spaces in LATITUD column
.withColumn("Lat_Ori", substring(col("LATITUDE"),-1,1)) # Get N,S,W,E Orientation from latitud
.withColumn("Long_Ori", substring(col("LONGITUDE"),-1,1)) # Get N,S,W,E Orientation from longitude
.withColumn("LATITUDE_CORRECTED", #THIS NEW COLUMN CORRECT THE COORINDATES DEPENDING ON THE ORIENTATION N,S,W,E
            when(col("Lat_Ori") == 'S', expr("substring(LATITUDE,1,length(LATITUDE) - 1 )") * - 1) #GET CORRECT COORDINATES
            .when(col("Lat_Ori") == 'N', expr("substring(LATITUDE,1,length(LATITUDE) - 1 )")) #GET CORRECT COORDINATES
            .when(col("Lat_Ori") == 'E', expr("substring(LATITUDE,1,length(LATITUDE) - 1 )") * -1) #GET CORRECT COORDINATES
            .otherwise(999.999) # ID FOR CHECKING IF SOME VALUE ISN'T TAKEN INTO ACCOUNT
)
.withColumn("LONGITUDE_CORRECTED", #THIS NEW COLUMN CORRECT THE COORINDATES DEPENDING ON THE ORIENTATION N,S,W,E
            when(col("Long_Ori") == 'E', expr("substring(LONGITUDE,1,length(LONGITUDE) - 1 )")) #GET CORRECT COORDINATES
            .when(col("Long_Ori") == 'W', expr("substring(LONGITUDE,1,length(LONGITUDE) - 1 )") * -1)#GET CORRECT COORDINATES
            .when(col("Lat_Ori") == 'N', expr("substring(LATITUDE,1,length(LATITUDE) - 1 )") * -1) #GET CORRECT COORDINATES
            .otherwise(999.999) # ID FOR CHECKING IF SOME VALUE ISN'T TAKEN INTO ACCOUNT
)
.select("COUNTRY","PORT","LATITUDE_CORRECTED","LONGITUDE_CORRECTED") # SELECT COUNTRIES OF INTEREST
)

## OBJECTIVE - DATA ANALYSIS:

To start doing some analysis with our data. We have to do a clear explanaition of our objective. Our Objective is:

**Detect and Precidit possible complications in port related to the Transpacific Route**

First, we need to identify the news that could be related to Ports inside the Transpacific Route. The countries we select to analyze are: Canada, United States, China and Japan. and for each country we select the three more importants ports that are useden in the Transpacific Route. The following list is a compilation per country that discribes: 
* Name of Country
* Code for Country
* Name of Port
* Location of the Port
* Code of the Location of the Port

**For further reference in locations you can check the following summary**

**CANADA**
Code for country = 'CA'

1. Port of Vancouver - British Columbia (CA02)
2. Puerto de Prince Rupert - Columbia Británica (CA02)
3. Port of Montreal - Quebec (CA10)

**USA**
Code for country = 'US'

1. Port of Los Angeles - California (USCA)
2. Port of Long Beach - California (USCA)
3. Port of Oakland - California (USCA)

**CHINA**
Code for country = 'CH'

1. Port of Shanghai - Shanghai (CH23)
2. Port of Shenzhen - Guangdong Province (CH30)
3. Port of Ningbo-Zhoushan - Zhejiang Province (CH02)

**JAPAN**
Code for Country = 'JA'

1. Port of Tokyo - Tokyo (JA40)
2. Port of Yokohama - Kanagawa Prefecture (JA19)
3. Port of Nagoya - Aichi Prefecture (JA01)

## Principal Cleaning

The table that we are going to use as a base is the GKG table. Before doing anything we need to clarify the basic steps of cleaning that this table would requiere:
1. Creation of a Date column of Type Date
2. Filtering news from a static period of time. In this case from January 2022 to July 2024
3. LOCATIONS column is delimited with '#', By splitting this column we can extract Country Code and Location Code
4. TONE column is delimited with ",", by splitting this column we can extract: Average tone of the new, Positive Score of the new, Negative Score from the new, and Polarity of the New
5. Filter news related to the Locations we are interested (In this case locations with the selected ports from the transpacific route)

In [0]:
GKG_PRINCIPAL_CLEANING =(GKG
.withColumn("Date", to_date(col("DATE"), "yyyyMMdd")) # CREATE DATE COLUMN
.filter("Date >= '2022-01-01' and Date < '2024-08-01'") # SELECTE NEWS FROM 2022 TO JULY 2024
.withColumn("CountryCode", split(col("LOCATIONS"),"#").getItem(2)) # GET COUNTRY CODE
.withColumn("LocationCode", split(col("LOCATIONS"),"#").getItem(3)) # GET LOCATION CODE
.withColumn("AverageTone", split(col("TONE"),",").getItem(0)) # GET AVERAGE TONE
.withColumn("TonePositiveScore", split(col("TONE"),",").getItem(1)) # GET TONE POSITIVE SCORE
.withColumn("ToneNegativeScore", split(col("TONE"),",").getItem(2)) # GET TONE NEGATIVE SCORE
.withColumn("Polarity", split(col("TONE"),",").getItem(3))  # GET TONE POLARITY
.filter(col("LocationCode").isin("CA02","CA02","CA10","USCA","CH23",'CH30',"CH02","JA40","JA19","JA01")) # FILTER THE NEWS RELATED TO THE LOCATIONS OF THE PORT (CHECK DEFINITION IN THE CELL ABOVE FOR MORE DETAILS IN THE CODES)
)

In [0]:
TABLE_OF_DATES = (
GKG_PRINCIPAL_CLEANING
.select("Date").distinct()
)

## How to deal with news with some sort of emotions?

As a logistic company, we can't base our decisions in emotions or a interpretation of a situation with a lot of emotion charge to it. This could easily cause wrong interpretations and could generate serious problems to a company.

In GKG, with the TONE varible we could make some decissions by extracting the average tone and the polarity. 

In the GKG codebook, it states that news with high emotional charge can be identify following this rule: The Average Tone is very neutral (close to 0) and the Polarity is High.

So, now the question that arise is "how close to 0 to considere it neutral" and "what is high in polarity"
* We define that a neutral average tone is going to be between -0.5 and 0.5 
* A high polarity is a value higher than 9. This value was obtained by analyzing percentiles. This 9 is approximately the 85% percentile.

The following piece of code accomplish the task to eliminate Emotional Charged News.

In [0]:
GKG_NOT_EMOTIONAL_CHARE = (GKG_PRINCIPAL_CLEANING
.withColumn("Neutrality", when((col("AverageTone") >= -0.5) & (col("AverageTone") <= 0.5),1).otherwise(0)) # GET NEUTRALITY OF THE NEW (FLAG 1=NEUTRAL, 0=NOT NEUTRAL)
.withColumn("EC", when((col("Neutrality") == 1) & (col('Polarity') >= 9),1).otherwise(0)) #GET EMOTIONAL CHARGED FLAG (1=EMOTIONAL CHARGE,0=NOT EMOTIONAL CHARGED)
.filter("EC == 0") # GET ONLY THE NEWS THAT ARE NOT EMOTIONAL CHARGED
)

## Approach 1
#### Analyzing News Growth to Predecit and Increase of Themes that could be Dangerous for Port Activities

In [0]:

GKG_PORTS_OV_FIRST_APPROACH = (        
GKG_NOT_EMOTIONAL_CHARE
.withColumn("BaseNews", when((col("THEMES").like("%PORT%")) & (col("THEMES").like("%TRANSPORT%")) & (~col("THEMES").like("%AIRPORT%")),1)) # BASE FLAG NEWS THAT HAVE THEME PORT AND TRANSPORTATIONS AND NOT AIRPORTS
.filter("BaseNews == 1") # SELECT NEWS ONLY RELATED TO BASE NEWS FLAG
.withColumn("NewsWithTINFA", when(col("THEMES").like("%TRANSPORT_INFRASTRUCTURE%"),1)) # FLAG THAT THE NEWS HAS THE THEME TRANSPORT INFRASTRCTURE
.withColumn("NewsWithTRADE", when(col("THEMES").like("%TRADE%"),1)) # FLAG THAT THE NEWS HAS THE THEME TRADE 
.withColumn("NewsWithME", when(col("THEMES").like("%MACROECONOMIC%"),1)) # FLAG THAT THE NEWS HAS THE THEME MACROECONOMICS
.withColumn("NewsWithPS", when(col("THEMES").like("%PUBLIC_SECTOR%"),1)) # FLAG THAT THE NEWS HAS THE THEME PUBLIC SECTOR
.withColumn("NewsWithMI", when(col("THEMES").like("%MARITIME_INCIDENT%"),1)) # FLAG THAT THE NEWS HAS THE THEME MARITIME_INCIDIENT
.fillna(0) # FILL WITH 0 THE COLUMNS WHERE OCURRENCES ARE INEXISTENT
.withColumn("Total", col("NewsWithTINFA")+col("NewsWithTRADE")+col("NewsWithME") + col("NewsWithPS") + col("NewsWithMI")) # TOTAL NUMBER OF THEMES FOUND IN THE NEWS
.filter("AverageTone < 0") # FILTER NEWS THAT HAVE A NEGATIVE AVERAGE TONE
.groupby("Date","Total","LocationCode").agg(sum("NewsWithTINFA").alias("NewsWithTINFA"),sum("NewsWithME").alias("NewsWithME"),sum("NewsWithPS").alias("NewsWithPS"),sum("NewsWithMI").alias("NewsWithMI"),sum("Total").alias("TotalThemesFindings"),count("Total").alias("NumberOfNews"))
)

GKG_PORTS_OV_FIRST_APPROACH_CD = (
TABLE_OF_DATES.crossJoin(GKG_PORTS_OV_FIRST_APPROACH.select("Total","LocationCode").distinct())
)


GKG_PORTS_FIRST_APPROACH = (
GKG_PORTS_OV_FIRST_APPROACH_CD.join(GKG_PORTS_OV_FIRST_APPROACH, ["Date","Total","LocationCode"],"left")
.fillna(0)
.persist()
)

GKG_PORTS_FIRST_APPROACH.count()
#.withColumn("PastValue", lag(col("NumberOfNews"),1).over(Window.partitionBy("Total").orderBy("Date")))

#.withColumn("Crecimiento", (col("Conteo") - col("PastValue"))/col("PastValue"))
#)

In [0]:
display(GKG_PORTS_FIRST_APPROACH)

In [0]:
display(
GKG_PRINCIPAL_CLEANING
.withColumn("BaseNews", when((col("THEMES").like("%PORT%")) & (col("THEMES").like("%TRANSPORT%")) & (~col("THEMES").like("%AIRPORT%")),1)) # BASE FLAG NEWS THAT HAVE THEME PORT AND TRANSPORTATIONS AND NOT AIRPORTS
.filter("BaseNews == 1") # SELECT NEWS ONLY RELATED TO BASE NEWS FLAG
.filter("LocationCode == 'CA02'")
.filter("Date == '2024-07-23'")
)

In [0]:
display(
GKG_PORTS_FIRST_APPROACH
.withColumn("NumberOfNewsPastDay", lag(col("NumberOfNews"),1).over(Window.partitionBy("Total","LocationCode").orderBy("Date")))
.withColumn("GrowthNumberOfNewsPastDay", (col("NumberOfNews") - col("NumberOfNewsPastDay"))/col("NumberOfNewsPastDay"))
.filter("LocationCode == 'CA02'")
.withColumn("LagGrowthNumberOfNewsPastDay", lag(col("GrowthNumberOfNewsPastDay"),1).over(Window.partitionBy("Total","LocationCode").orderBy("Date")))
.fillna(0)
)

Databricks visualization. Run in Databricks to view.

## Aproach 2 - Adding Weights to the Ocurrences of Themes of Interest to express

In [0]:
GKG_PORTS_OV_SECOND_APPROACH = (    
GKG_NOT_EMOTIONAL_CHARE
.withColumn("BaseNews", when((col("THEMES").like("%PORT%")) & (col("THEMES").like("%TRANSPORT%") & (~col("THEMES").like("%AIRPORT%"))),1))
.filter("BaseNews == 1")
.withColumn("NewsWithTINFA", when(col("THEMES").like("%TRANSPORT_INFRASTRUCTURE%"),1))
.withColumn("NewsWithTRADE", when(col("THEMES").like("%TRADE%"),1))
.withColumn("NewsWithME", when(col("THEMES").like("%MACROECONOMIC%"),1))
.withColumn("NewsWithPS", when(col("THEMES").like("%PUBLIC_SECTOR%"),1))
.withColumn("NewsWithMI", when(col("THEMES").like("%MARITIME_INCIDENT%"),1))
.fillna(0)
.withColumn("Total", col("NewsWithTINFA")+col("NewsWithTRADE")+col("NewsWithME") + col("NewsWithPS") + col("NewsWithMI"))
.filter("AverageTone < 0")
.groupby("Date","Total","LocationCode").agg(count("Total").alias("NumberOfNews"))
.withColumn('WeightedCountOfNews', when(col("Total") == 5, col("NumberOfNews") * 500)
.when(col('Total') == 4, col("NumberOfNews") * 250)
.when(col("NumberOfNews") ==3, col("NumberOfNews") * 100)
.when(col("Total") == 2, col("NumberOfNews") * 5)
.otherwise(0))
)

GKG_PORTS_OV_SECOND_APPROACH_CD = (
TABLE_OF_DATES.crossJoin(GKG_PORTS_OV_SECOND_APPROACH.select("Total","LocationCode").distinct())
)

GKG_PORTS_SECOND_APPROACH = (
GKG_PORTS_OV_SECOND_APPROACH_CD.join(GKG_PORTS_OV_SECOND_APPROACH, ["Date","Total","LocationCode"],"left")
.fillna(0)
.persist()
)

GKG_PORTS_SECOND_APPROACH.count()

In [0]:
display(
GKG_PORTS_SECOND_APPROACH
.groupBy("Date","LocationCode").agg(sum("WeightedCountOfNews").alias("WeightedCountOfNews"))
.withColumn("LagWeightedCountNews", lag(col("WeightedCountOfNews"),2).over(Window.partitionBy("LocationCode").orderBy("Date")))
.fillna(0)
.filter(col("LocationCode").isin(
        'CA02',  # Port of Vancouver - British Columbia
        'CA10',  # Port of Montreal - Quebec
        'USCA',  # Port of Los Angeles, Long Beach, Oakland - California
        'CH23',  # Port of Shanghai - Shanghai
        'CH30',  # Port of Shenzhen - Guangdong Province
        'CH02',  # Port of Ningbo-Zhoushan - Zhejiang Province
        'JA40',  # Port of Tokyo - Tokyo
        'JA19',  # Port of Yokohama - Kanagawa Prefecture
        'JA01'  # Port of Nagoya - Aichi Prefecture
    ))
)

Databricks visualization. Run in Databricks to view.

In [0]:
while True:
    continue